In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
links = [
"https://www.caridestinasi.com/malaysia/destinasi-menarik-malaysia/",
"https://www.caridestinasi.com/malaysia/tempat-makan-best-malaysia/",
"https://www.caridestinasi.com/malaysia/tempat-penginapan-menarik-malaysia/",
"https://www.caridestinasi.com/malaysia/aktiviti-menarik-malaysia/"]
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 


In [3]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "html.parser")

    div = soup.find_all('p', attrs = {"class":"entry-title td-module-title"})
    
    if len(div) == 0:
        return

    for link in soup.find_all('p', attrs = {"class":"entry-title td-module-title"}):

        href = link.find('a').get('href')

        hrefs.append(href)
    

In [5]:
max_worker = 10


for url in links:
    
    print(url)
    r = requests.get(url,headers = headers)
    soup = BeautifulSoup(r.text, "html.parser")
    nav = soup.find('div',attrs={'class':'page-nav td-pb-padding-side'})
    a = nav.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page/')[1].replace('/','')) for a_ in pgno])
    print(max_page)

    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()


https://www.caridestinasi.com/malaysia/destinasi-menarik-malaysia/
25


100%|██████████| 3/3 [00:10<00:00,  3.60s/it]


https://www.caridestinasi.com/malaysia/tempat-makan-best-malaysia/
11


100%|██████████| 2/2 [00:05<00:00,  2.90s/it]


https://www.caridestinasi.com/malaysia/tempat-penginapan-menarik-malaysia/
16


100%|██████████| 2/2 [00:06<00:00,  3.20s/it]


https://www.caridestinasi.com/malaysia/aktiviti-menarik-malaysia/
9


100%|██████████| 1/1 [00:03<00:00,  3.27s/it]


In [8]:
len(hrefs)

470

In [9]:
with open(f'caridestinasi-link.json', 'a') as f:
    json.dump(hrefs, f)

In [4]:
with open('caridestinasi-link.json') as fopen:
    hrefs = json.load(fopen)

In [6]:
hrefs = list(set(hrefs))

In [11]:
for x in tqdm(hrefs):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)

    soup = BeautifulSoup(r.text, "html.parser")

    try:
        headline = soup.find('h1', attrs = {'class':"tdb-title-text"}).text
        h = soup.find('div', attrs = {'class':"tdb-block-inner td-fix-index"})
        p = [p.text for p in h.find_all(['p','h2','h3']) if p.text and len(p.text.split()) > 3]
        print(p)
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        continue


    data = {'url': x, 'headline': headline, 'content': p}


    with open('caridestinasi.jsonl', 'a') as f:
        json.dump(data, f)
        f.write('\n')

  0%|          | 2/470 [00:00<01:42,  4.57it/s]

[]
[]


  1%|          | 4/470 [00:00<01:24,  5.48it/s]

[]
[]


  1%|▏         | 6/470 [00:01<01:23,  5.55it/s]

...

[]


  2%|▏         | 10/470 [00:01<01:31,  5.00it/s]

[]
[]


  2%|▏         | 10/470 [00:02<01:36,  4.77it/s]


KeyboardInterrupt: 